# Erdős-Rényi and Barabási-Albert Networks

In this exercise, we generate (i) Erdős-Rényi and (ii) Barabási-Albert networks and analyze some of their properties.

To build an Erdős-Rényi network, begin with a set of $N$ isolated nodes and iterate over all possible node pairs. Two nodes are connected with probability $p$.

Barabási-Albert networks are constructed via preferential attachment after starting from a connected network with $m_{0}$ nodes (e.g., a dyad). Preferential attachment means that a new node is connected to $m \leq m_{0}$ existing nodes and the probability $p_{i}$ of being connected to an existing node $i$ is proportional to the degree $k_{i}$ of that node (i.e., $p_{i}=k_{i} / \sum_{j} k_{j}$).

(a) You can start with networks that have $N=100$ nodes and later extend your analyses to networks with 1000 and 10000 nodes. Vary the connection probability $p$ for Erdős-Rényi networks and the parameter $m$ for Barabási-Albert networks. What is the influence of these variations on the degree distribution?

**Answer:**

In [ ]:
import warnings

import altair as alt
import networkx as nx
import pandas as pd

warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
def plot_erdos_renyi_degree(n, p):
    """
    Plot degree distribution of Erdős-Rényi networks given n and p.
    """
    G = nx.erdos_renyi_graph(n, p)
    degree_sequence = pd.DataFrame({"degree_sequence": [d for n, d in G.degree()]})
    return alt.Chart(degree_sequence).mark_bar().encode(
        x= alt.X("degree_sequence:Q", bin=alt.Bin(maxbins=15), title="degree_sequence"),
        y=alt.Y("count()", title="frequency"),
    )

def plot_barabasi_albert_degree(n, m):
    """
    Plot degree distribution of Barabási-Albert networks given n and m.
    """
    G = nx.barabasi_albert_graph(n, m)
    degree_sequence = pd.DataFrame({"degree_sequence": [d for n, d in G.degree()]})
    return alt.Chart(degree_sequence).mark_bar().encode(
        x= alt.X("degree_sequence:Q", bin=alt.Bin(maxbins=15), title="degree_sequence"),
        y=alt.Y("count()", title="frequency"),
    )

In [ ]:
plot_erdos_renyi_degree(n=1000, p=0.1)

In [ ]:
plot_erdos_renyi_degree(n=1000, p=0.3)

In [ ]:
plot_erdos_renyi_degree(n=1000, p=0.8)

Higher $p$ results in higher number of degrees for each node for Erdős-Rényi networks.

In [ ]:
plot_barabasi_albert_degree(n=1000, m=100)

In [ ]:
plot_barabasi_albert_degree(n=1000, m=300)

In [ ]:
plot_barabasi_albert_degree(n=1000, m=800)

Higher $m$ results in more poloarized degree distribution for Barabási-Albert networks.

(b) Implement Dijkstra's algorithm and determine the average path length between two nodes for networks (i) and (ii) as a function of the number of nodes $N$. What do you observe?

**Answer:**

In [ ]:
def calculate_erdos_renyi_length(n):
    """
    Calculate the average path length between two nodes for Erdős-Rényi networks given n.
    """
    G = nx.erdos_renyi_graph(n, p=0.5)
    return nx.average_shortest_path_length(G)

def calculate_barabasi_albert_length(n):
    """
    Calculate the average path length between two nodes for Barabási-Albert networks given n.
    """
    G = nx.barabasi_albert_graph(n, m=100)
    return nx.average_shortest_path_length(G)

def plot_erdos_renyi_length():
    """
    Plot the average path length between two nodes for Erdős-Rényi networks as a function of n.
    """
    source = pd.DataFrame(
        {
            "n": range(100, 1000, 50),
            "length": [calculate_erdos_renyi_length(n) for n in range(100, 1000, 50)],
        }
    )
    return alt.Chart(source).mark_line().encode(
        x = "n",
        y = "length"
    )

def plot_barabasi_albert_length():
    """
    Plot the average path length between two nodes for Barabási-Albert networks as a function of n.
    """
    source = pd.DataFrame(
        {
            "n": range(101, 1001, 10),
            "length": [calculate_barabasi_albert_length(n) for n in range(101, 1001, 10)],
        }
    )
    return alt.Chart(source).mark_line().encode(
        x = "n",
        y = "length"
    )

In [ ]:
plot_erdos_renyi_length()

The average length is almost the same for different values of $n$ in Erdős-Rényi networks.

In [ ]:
plot_barabasi_albert_length()

As $n$ increase, the average length first decreases then increases in Barabási-Albert networks.

# Visualizing Networks with graph-tool

This exercise will provide a short introduction to working with the network analysis and visualization library graph-tool. (A commonly used alternative is the library NetworkX.)

(a) Create a Delaunay triangulation network and its minimum spanning tree using the following commands.

**Answer:**

In [ ]:
from graph_tool.centrality import betweenness
from graph_tool.draw import graph_draw, prop_to_size
from graph_tool.generation import triangulation
from graph_tool.topology import min_spanning_tree
from matplotlib.cm import gist_heat
from numpy.random import random
import graph_tool as gt

In [ ]:
g, pos = triangulation(random((500, 2)) * 4, type="delaunay")
tree = min_spanning_tree(g)

(b) Visualize the resulting network that contains only those nodes that belong to the minimum spanning tree.

**Answer:**

In [ ]:
# Refer to documentation on minimum spanning tree:
# https://graph-tool.skewed.de/static/doc/topology.html#graph_tool.topology.min_spanning_tree
u = gt.GraphView(g, efilt=tree)
graph_draw(u, pos=pos);

(c) Compute geodesic edge and node betweennness centralities and again visualize the network. Color nodes and use edge widths according to their centralities. (Edge with large centrality should have a large edge width in your plots.)

**Answer:**

In [ ]:
vp, ep = betweenness(g)
# Refer to documentation on betweenness:
# https://graph-tool.skewed.de/static/doc/centrality.html#graph_tool.centrality.betweenness
graph_draw(
    g,
    pos,
    vertex_fill_color=vp,
    vertex_size=prop_to_size(vp, mi=5, ma=15),
    edge_pen_width=prop_to_size(ep, mi=0.5, ma=5),
    vcmap=gist_heat,
    vorder=vp,
);